In [1]:
# coding: utf-8
import pandas as pd
import numpy as np

In [2]:
year = 2018
keyword = 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesForeign'

# Load data
if (year>2009):
    q1_sub = pd.read_csv(str(year)+'q1sub.tsv',sep='\t', encoding = "ISO-8859-1")
    q1_num = pd.read_csv(str(year)+'q1num.tsv',sep='\t', encoding = "ISO-8859-1")
    
q2_sub = pd.read_csv(str(year)+'q2sub.tsv',sep='\t', encoding = "ISO-8859-1")
q2_num = pd.read_csv(str(year)+'q2num.tsv',sep='\t', encoding = "ISO-8859-1")
q3_sub = pd.read_csv(str(year)+'q3sub.tsv',sep='\t', encoding = "ISO-8859-1")
q3_num = pd.read_csv(str(year)+'q3num.tsv',sep='\t', encoding = "ISO-8859-1")
q4_sub = pd.read_csv(str(year)+'q4sub.tsv',sep='\t', encoding = "ISO-8859-1")
q4_num = pd.read_csv(str(year)+'q4num.tsv',sep='\t', encoding = "ISO-8859-1")

sub=q2_sub.append(q3_sub).append(q4_sub)
num=q2_num.append(q3_num).append(q4_num)

if (year>2009):
    sub=sub.append(q1_sub)
    num=num.append(q1_num)

sub = q1_sub
num = q1_num

field_list = ['Assets','Liabilities','LiabilitiesAndStockholdersEquity',
                                    'IncomeLossFromContinuingOperationsBeforeIncomeTaxesForeign',
                'IncomeLossFromContinuingOperationsBeforeIncomeTaxesDomestic',
                 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest']

In [3]:
# Including only the fields that we want
num = num[num.tag.isin([keyword])]
num_10q = num.copy()

# Adding cik to num file
num_10q = num_10q.merge(sub.loc[:,['adsh','cik','form']],on='adsh',how='left')
num = num.rename(columns={'ddate':'period'})
num_10q = num_10q.rename(columns={'ddate':'period'})

# Type of records we want
sub = sub[sub.form.isin(['10-K'])]
sub.adsh.nunique()
# sub[sub.cik.isin(['1800'])][['adsh','cik','period','form']]

3944

In [4]:
## STEP #1:
# Get values from NUM file using SUM file adsh,period
df = pd.merge(sub,num,on=['adsh','period'],how='left')
df = df.loc[:,['adsh','cik','name','sic','coreg','fye','fy','qtrs','fp','filed','form','period','nciks','aciks','tag','value']]
df.sort_values('cik',inplace=True)
# df[df.cik.isin(['1132143'])][['adsh','cik','period','form','tag']]

In [5]:
# Setting unique identifiers for pivoting table
df['coreg'] = df['coreg'].fillna('null')
df['value'] = df['value'].astype('str')
df['adsh+coreg+value'] = df['adsh'] + "_"+ df['coreg'] + "_" + df['value']
# Reset data type numeric for value
df['value'] = df['value'].astype('float64')

# Make a copy for pivoting table
data = df.copy()
data.sort_values(['cik','tag','period'],inplace=True)
data.adsh.nunique()

3944

In [6]:
# Pivoting table
fin_data =data.loc[:,['adsh+coreg+value','adsh','cik','name','sic','coreg','fy','fp','form','filed','period','nciks','aciks']].\
merge(pd.pivot_table(data,index=data['adsh+coreg+value'],columns='tag',values='value').reset_index(),on='adsh+coreg+value',how='left')

# Some statistics and info
fin_data.adsh.nunique()

3944

In [7]:
fin_data[fin_data.coreg!='null'].groupby('cik').ngroups

9

In [8]:
fin_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3995 entries, 0 to 3994
Data columns (total 14 columns):
adsh+coreg+value                                              3995 non-null object
adsh                                                          3995 non-null object
cik                                                           3995 non-null int64
name                                                          3995 non-null object
sic                                                           3994 non-null float64
coreg                                                         3995 non-null object
fy                                                            3995 non-null int64
fp                                                            3995 non-null object
form                                                          3995 non-null object
filed                                                         3995 non-null int64
period                                                        399

In [9]:
## STEP #2:
# Look for value of records were we couldn't get values by matching (adsh,period) from SUB file to NUM file
# Look for this values matching (cik,period) instead

if keyword in fin_data.columns.tolist():
    missing_10K = num_10q.merge(fin_data[fin_data[keyword].isnull()].loc[:,['name','cik','sic','period','fy','qtrs']],
                                 on=['cik','period'],how='inner').\
                                 drop_duplicates(['adsh','period','tag','value','coreg']).sort_values('cik')
        
else:
    fin_fin_data = fin_data.copy()
    fin_fin_data[keyword] = np.nan
    fin_fin_data.sort_values(['cik','coreg','period','form'],inplace=True)
    fin_fin_data.drop_duplicates(['adsh+coreg+value','period','form'],inplace=True)
    fin_fin_data.to_csv('josh_data'+str(year)+keyword+'.csv')

In [11]:
if (keyword in fin_data.columns.tolist()) and (len(missing_10K) > 1):
    # Setting unique identifiers for pivoting table
    missing_10K['coreg'] = missing_10K['coreg'].fillna('null')
    missing_10K['value'] = missing_10K['value'].astype('str')
    missing_10K['adsh+coreg+value'] = missing_10K['adsh'] + "_"+ missing_10K['coreg'] + "_" + missing_10K['value']
    
    # Reset data type numeric for value
    missing_10K['value'] = missing_10K['value'].astype('float64')

    # Pivoting table
    missing_10K = missing_10K.loc[:,['adsh+coreg+value','adsh','cik','name','sic','coreg','fye','fy','fp','form','period','nciks','aciks']].\
    merge(pd.pivot_table(missing_10K,index=missing_10K['adsh+coreg+value'],columns='tag',values='value').\
          reset_index(),on='adsh+coreg+value',how='left').drop_duplicates(['adsh','cik','fy','coreg',keyword],keep='last')
    
    missing_10K[missing_10K.cik.isin(['1132143'])][['adsh','cik','period','form',keyword]]

KeyError: 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesForeign'

In [ ]:
## RESULTS
if keyword in fin_data.columns.tolist():
    fin_fin_data = fin_data.append(missing_10K).sort_values('cik')
    fin_fin_data.sort_values(['cik','coreg','period','form'],inplace=True)

    # Keep only non-null values
    # fin_fin_data = fin_fin_data[fin_fin_data[keyword].notnull()]

    # Get rid of duplicate records
    fin_fin_data.drop_duplicates(['adsh+coreg+value','period','form'],inplace=True)
    fin_fin_data.info()

    ## STORING RESULTS
    fin_fin_data.to_csv('josh_data'+str(year)+keyword+'.csv')